In [1]:
%%capture
!pip install --upgrade -tensorflow_hub
# !pip install -U -huggingface_hub

import textattack
import transformers
import torch
import time
from datasets import Dataset
import sys
import hashlib
import numpy as np

from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertForMaskedLM, pipeline
from textattack.attack_recipes import (
    TextBuggerLi2018, DeepWordBugGao2018, TextFoolerJin2019, BERTAttackLi2020
)
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder
from textattack.models.wrappers import ModelWrapper

sys.path.append('../')
from eval_utils import *
sys.path.pop()

2023-08-02 15:41:25.326223: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-02 15:41:26.415076: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# set a seed, because reproducability is cool
np.random.seed(int(hashlib.sha256('Harrison Gietz'.encode('utf-8')).hexdigest(), 16) % 2**32)
torch.cuda.empty_cache()

device = input('enter a device name to run on: ')

ag_tokenizer = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-ag-news")
ag_model = AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-ag-news")
ag_model.to(device)
ag_pipeline = pipeline('sentiment-analysis', model=ag_model, tokenizer=ag_tokenizer)
ag_pipeline.device = next(ag_model.parameters()).device

ag_model_directory = "../../../models/bert-uncased_maskedlm_agnews_july31" #first diff
finetuned_ag_maskedlm = BertForMaskedLM.from_pretrained(ag_model_directory)
finetuned_ag_maskedlm.to(device)
ag_fill_mask = pipeline("fill-mask", model=finetuned_ag_maskedlm, tokenizer=ag_tokenizer)
ag_fill_mask.device = next(ag_model.parameters()).device

num_voter = 11
mask_pct = 0.3    
    
attack = TextFoolerJin2019

dataset_val = input('Enter the number of samples to run on (100 or 1000): ')

if dataset_val == '100':
    loaded_ag_100 = Dataset.load_from_disk('../data/filtered_ag_clean_100')
    ag_100 = textattack.datasets.Dataset(convert_to_tuples(loaded_ag_100))
    dataset = ag_100
    dataset_name = 'ag-news100'
elif dataset_val =='1000':
    loaded_ag_1000 = Dataset.load_from_disk('../data/filtered_ag_clean_1000')
    ag_1000 = textattack.datasets.Dataset(convert_to_tuples(loaded_ag_1000))
    dataset = ag_1000
    dataset_name = 'ag-news1000'
else:
    raise ValueError('Number of samples not supported')
    
defense = input('Specify a defense type among "default", "logit", "maj_log", "one_hot": ')
if defense == "default":
    ag_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(ag_model, ag_tokenizer)
    print(ag_wrapper)
elif defense == "logit":
    ag_wrapper = MaskDemaskWrapper(ag_model, ag_tokenizer, ag_fill_mask, num_voter, mask_pct, 'logit')
elif defense == 'maj_log':
    ag_wrapper = MaskDemaskWrapper(ag_model, ag_tokenizer, ag_fill_mask, num_voter, mask_pct, 'maj_log')
elif defense == "one_hot":
    ag_wrapper = MaskDemaskWrapper(ag_model, ag_tokenizer, ag_fill_mask, num_voter, mask_pct, 'maj_one_hot')
else:
    raise ValueError('Not a valid defense type.')
    
print(f'using num_voter = {num_voter} and mask_pct = {mask_pct} with dataset = {dataset_name}...')

# Parse the attack name
attack_name = parse_attack_name(attack)
attack = attack.build(ag_wrapper)

cand_size = int(input('enter number of candidates (recommended 12 for quicker run, 50 otherwise): '))
# change candidate size
attack.transformation.max_candidates = cand_size
# adjust attack threshold to match Li et al. 2023 (0.5 theshold for AgNews Universal sentences encoder):
attack.constraints[2] = UniversalSentenceEncoder(metric = 'angular', threshold = 0.5, 
                                                 window_size = 15, skip_text_shorter_than_window=True, 
                                                 compare_against_original=False)

# Set up arguments for the attack
attack_args = textattack.AttackArgs(
    num_examples=len(dataset),
    log_to_csv=f'{attack_name}_{dataset_name}_candsize{cand_size}_mp{mask_pct}_nv{num_voter}_{defense}_log.csv',
    checkpoint_interval=25, 
    checkpoint_dir="chkpts_2", 
    disable_stdout=True
)
# Perform the attack and save the results
attacker = textattack.Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

print(f'The above are results for {attack_name}_{dataset_name}_candsize{cand_size}_mp{mask_pct}_nv{num_voter}_{defense}.')

enter a device name to run on: cuda:2
Enter the number of samples to run on (100 or 1000): 1000
Specify a defense type among "default", "logit", "maj_log", "one_hot": maj_log
using num_voter = 11 and mask_pct = 0.3 with dataset = ag-news1000...
enter number of candidates (recommended 12 for quicker run, 50 otherwise): 12
Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  12
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angu


  0%|          | 0/1000 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
2023-08-02 15:42:49.481734: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-08-02 15:42:51.524310: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype string
	 [[{{node inputs}}]]

  0%|          | 1/1000 [02:04<34:34:1


[Succeeded / Failed / Skipped / Total] 3 / 19 / 3 / 25:   3%|▎         | 26/1000 [44:28<27:45:54, 102.62s/it]
[Succeeded / Failed / Skipped / Total] 3 / 20 / 3 / 26:   3%|▎         | 26/1000 [44:28<27:45:55, 102.62s/it]
[Succeeded / Failed / Skipped / Total] 3 / 20 / 3 / 26:   3%|▎         | 27/1000 [44:56<26:59:43, 99.88s/it] 
[Succeeded / Failed / Skipped / Total] 4 / 20 / 3 / 27:   3%|▎         | 27/1000 [44:56<26:59:44, 99.88s/it]
[Succeeded / Failed / Skipped / Total] 4 / 20 / 3 / 27:   3%|▎         | 28/1000 [46:16<26:46:26, 99.16s/it]
[Succeeded / Failed / Skipped / Total] 4 / 21 / 3 / 28:   3%|▎         | 28/1000 [46:16<26:46:27, 99.16s/it]
[Succeeded / Failed / Skipped / Total] 4 / 21 / 3 / 28:   3%|▎         | 29/1000 [46:17<25:49:45, 95.76s/it]
[Succeeded / Failed / Skipped / Total] 4 / 21 / 4 / 29:   3%|▎         | 29/1000 [46:17<25:49:46, 95.76s/it]
[Succeeded / Failed / Skipped / Total] 4 / 21 / 4 / 29:   3%|▎         | 30/1000 [47:30<25:35:56, 95.01s/it]
[Succeeded / Fa


[Succeeded / Failed / Skipped / Total] 7 / 37 / 6 / 50:   5%|▌         | 51/1000 [1:17:36<24:04:12, 91.31s/it]
[Succeeded / Failed / Skipped / Total] 7 / 38 / 6 / 51:   5%|▌         | 51/1000 [1:17:36<24:04:12, 91.31s/it]
[Succeeded / Failed / Skipped / Total] 7 / 38 / 6 / 51:   5%|▌         | 52/1000 [1:20:02<24:19:08, 92.35s/it]
[Succeeded / Failed / Skipped / Total] 7 / 39 / 6 / 52:   5%|▌         | 52/1000 [1:20:02<24:19:08, 92.35s/it]
[Succeeded / Failed / Skipped / Total] 7 / 39 / 6 / 52:   5%|▌         | 53/1000 [1:22:02<24:25:53, 92.88s/it]
[Succeeded / Failed / Skipped / Total] 7 / 40 / 6 / 53:   5%|▌         | 53/1000 [1:22:02<24:25:54, 92.88s/it]
[Succeeded / Failed / Skipped / Total] 7 / 40 / 6 / 53:   5%|▌         | 54/1000 [1:23:01<24:14:29, 92.25s/it]
[Succeeded / Failed / Skipped / Total] 8 / 40 / 6 / 54:   5%|▌         | 54/1000 [1:23:01<24:14:29, 92.25s/it]
[Succeeded / Failed / Skipped / Total] 8 / 40 / 6 / 54:   6%|▌         | 55/1000 [1:25:07<24:22:36, 92.86s/it]



[Succeeded / Failed / Skipped / Total] 13 / 53 / 9 / 75:   8%|▊         | 76/1000 [1:55:06<23:19:30, 90.88s/it]
[Succeeded / Failed / Skipped / Total] 13 / 54 / 9 / 76:   8%|▊         | 76/1000 [1:55:06<23:19:31, 90.88s/it]
[Succeeded / Failed / Skipped / Total] 13 / 54 / 9 / 76:   8%|▊         | 77/1000 [1:57:05<23:23:30, 91.24s/it]
[Succeeded / Failed / Skipped / Total] 13 / 55 / 9 / 77:   8%|▊         | 77/1000 [1:57:05<23:23:31, 91.24s/it]
[Succeeded / Failed / Skipped / Total] 13 / 55 / 9 / 77:   8%|▊         | 78/1000 [1:59:25<23:31:43, 91.87s/it]
[Succeeded / Failed / Skipped / Total] 13 / 56 / 9 / 78:   8%|▊         | 78/1000 [1:59:25<23:31:43, 91.87s/it]
[Succeeded / Failed / Skipped / Total] 13 / 56 / 9 / 78:   8%|▊         | 79/1000 [2:01:09<23:32:30, 92.02s/it]
[Succeeded / Failed / Skipped / Total] 13 / 57 / 9 / 79:   8%|▊         | 79/1000 [2:01:09<23:32:30, 92.02s/it]
[Succeeded / Failed / Skipped / Total] 13 / 57 / 9 / 79:   8%|▊         | 80/1000 [2:03:26<23:39:30, 92

...


[Succeeded / Failed / Skipped / Total] 186 / 701 / 38 / 925:  93%|█████████▎| 926/1000 [27:56:50<2:14:00, 108.65s/it]
[Succeeded / Failed / Skipped / Total] 186 / 702 / 38 / 926:  93%|█████████▎| 926/1000 [27:56:50<2:14:00, 108.65s/it]
[Succeeded / Failed / Skipped / Total] 186 / 702 / 38 / 926:  93%|█████████▎| 927/1000 [27:59:23<2:12:15, 108.70s/it]
[Succeeded / Failed / Skipped / Total] 186 / 703 / 38 / 927:  93%|█████████▎| 927/1000 [27:59:23<2:12:15, 108.70s/it]
[Succeeded / Failed / Skipped / Total] 186 / 703 / 38 / 927:  93%|█████████▎| 928/1000 [28:01:42<2:10:28, 108.73s/it]
[Succeeded / Failed / Skipped / Total] 186 / 704 / 38 / 928:  93%|█████████▎| 928/1000 [28:01:42<2:10:28, 108.73s/it]
[Succeeded / Failed / Skipped / Total] 186 / 704 / 38 / 928:  93%|█████████▎| 929/1000 [28:03:35<2:08:40, 108.74s/it]
[Succeeded / Failed / Skipped / Total] 186 / 705 / 38 / 929:  93%|█████████▎| 929/1000 [28:03:35<2:08:40, 108.74s/it]
[Succeeded / Failed / Skipped / Total] 186 / 705 / 38 /


[Succeeded / Failed / Skipped / Total] 190 / 721 / 39 / 950:  95%|█████████▌| 951/1000 [28:48:51<1:29:04, 109.08s/it]
[Succeeded / Failed / Skipped / Total] 190 / 722 / 39 / 951:  95%|█████████▌| 951/1000 [28:48:51<1:29:04, 109.08s/it]
[Succeeded / Failed / Skipped / Total] 190 / 722 / 39 / 951:  95%|█████████▌| 952/1000 [28:52:23<1:27:20, 109.18s/it]
[Succeeded / Failed / Skipped / Total] 190 / 723 / 39 / 952:  95%|█████████▌| 952/1000 [28:52:23<1:27:20, 109.18s/it]
[Succeeded / Failed / Skipped / Total] 190 / 723 / 39 / 952:  95%|█████████▌| 953/1000 [28:53:58<1:25:30, 109.17s/it]
[Succeeded / Failed / Skipped / Total] 190 / 724 / 39 / 953:  95%|█████████▌| 953/1000 [28:53:58<1:25:30, 109.17s/it]
[Succeeded / Failed / Skipped / Total] 190 / 724 / 39 / 953:  95%|█████████▌| 954/1000 [28:55:17<1:23:40, 109.14s/it]
[Succeeded / Failed / Skipped / Total] 191 / 724 / 39 / 954:  95%|█████████▌| 954/1000 [28:55:17<1:23:40, 109.14s/it]
[Succeeded / Failed / Skipped / Total] 191 / 724 / 39 /


[Succeeded / Failed / Skipped / Total] 195 / 741 / 39 / 975:  98%|█████████▊| 976/1000 [29:35:44<43:39, 109.16s/it]
[Succeeded / Failed / Skipped / Total] 195 / 742 / 39 / 976:  98%|█████████▊| 976/1000 [29:35:45<43:39, 109.16s/it]
[Succeeded / Failed / Skipped / Total] 195 / 742 / 39 / 976:  98%|█████████▊| 977/1000 [29:37:59<41:51, 109.19s/it]
[Succeeded / Failed / Skipped / Total] 195 / 743 / 39 / 977:  98%|█████████▊| 977/1000 [29:37:59<41:51, 109.19s/it]
[Succeeded / Failed / Skipped / Total] 195 / 743 / 39 / 977:  98%|█████████▊| 978/1000 [29:40:19<40:02, 109.22s/it]
[Succeeded / Failed / Skipped / Total] 195 / 744 / 39 / 978:  98%|█████████▊| 978/1000 [29:40:19<40:02, 109.22s/it]
[Succeeded / Failed / Skipped / Total] 195 / 744 / 39 / 978:  98%|█████████▊| 979/1000 [29:40:19<38:11, 109.11s/it]
[Succeeded / Failed / Skipped / Total] 195 / 744 / 40 / 979:  98%|█████████▊| 979/1000 [29:40:19<38:11, 109.11s/it]
[Succeeded / Failed / Skipped / Total] 195 / 744 / 40 / 979:  98%|█████

[Succeeded / Failed / Skipped / Total] 197 / 763 / 40 / 1000: 100%|██████████| 1000/1000 [30:23:30<00:00, 109.41s/it]



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 197    |
| Number of failed attacks:     | 763    |
| Number of skipped attacks:    | 40     |
| Original accuracy:            | 96.0%  |
| Accuracy under attack:        | 76.3%  |
| Attack success rate:          | 20.52% |
| Average perturbed word %:     | 8.08%  |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 221.32 |
+-------------------------------+--------+
The above are results for TextFoolerJin2019_ag-news1000_candsize12_mp0.3_nv11_maj_log.
